In [1]:
import cv2
import numpy as np
import mediapipe as mp
import pynput
import time
import pygame

pygame 2.6.1 (SDL 2.28.4, Python 3.11.11)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
pose = mp_pose.Pose(static_image_mode=False)

keyboard = pynput.keyboard.Controller()
mouse = pynput.mouse.Controller()

record_events = False
countdown_started = False
countdown_start_time = None

In [3]:
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    ab = a - b
    cb = c - b

    dot_product = np.dot(ab, cb)
    magnitude_ab = np.linalg.norm(ab)
    magnitude_cb = np.linalg.norm(cb)

    angle = np.arccos(dot_product / (magnitude_ab * magnitude_cb))
    return np.degrees(angle)

def play_sound(file):
    pygame.mixer.init()
    pygame.mixer.music.load(file)
    pygame.mixer.music.play()

def calibrate_angles():
    calibration_data = {"front": [], "right": [], "up": [], "left": [], "down": []}
    directions = ["front", "up", "down", "right", "left"]
    calibration_results = {}

    print("Presiona 'SPACE' para comenzar la cuenta atrás.")
    while True:
        ret, frame = cap.read()
        overlay = frame.copy()
        cv2.rectangle(overlay, (15, 15), (550, 50), (0, 0, 0), thickness=cv2.FILLED)
        cv2.addWeighted(overlay, 0.5, frame, 0.5, 0, frame)
        cv2.putText(frame, "Presiona 'SPACE' para empezar a calibrar", (35, 40), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 255, 255), 2)
        cv2.imshow("Calibration", frame)
        if cv2.waitKey(1) & 0xFF == 32:
            break
    
    time_difference = 0
    start_time = time.time()

    text = "PREPARATE PARA LA CALIBRACION"
    text_size = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 1, 2)[0]

    available = 0
    play_sound("music.mp3")
    while time.time() - start_time < 10:
        time_difference = time.time() - start_time
        ret, frame = cap.read()
        if not ret:
            break
        
        height, width, _ = frame.shape
        text_x = (width - text_size[0]) // 2
        text_y = height // 2

        bar_x = (width - 500) // 2
        bar_y = text_y + 30

        if available > 0:
            dif = cv2.absdiff(frame, pframe)        
            cv2.putText(dif, text, (text_x, text_y), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
            draw_countdown_progress_bar(dif, bar_x, bar_y, 500, 50, time_difference, 10, (0, 255, 0), 1)
            cv2.imshow("Calibration", dif)
        else:
            available = 1
        pframe = frame.copy()

        if cv2.waitKey(1) & 0xFF == 27:
            break
        
    pygame.mixer.music.stop()
    for direction in directions:
        play_sound("ok.mp3")

        time_difference = 0
        start_time = time.time()
        while time.time() - start_time < 5:
            time_difference = time.time() - start_time
            ret, frame = cap.read()
            if not ret:
                break

            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = pose.process(frame_rgb)

            if results.pose_landmarks is not None:
                landmarks = results.pose_landmarks.landmark

                left_shoulder_coords = (
                    landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].x * frame.shape[1],
                    landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].y * frame.shape[0]
                )
                right_shoulder_coords = (
                    landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].x * frame.shape[1],
                    landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].y * frame.shape[0]
                )
                nose_coords = (
                    landmarks[mp_pose.PoseLandmark.NOSE].x * frame.shape[1],
                    landmarks[mp_pose.PoseLandmark.NOSE].y * frame.shape[0]
                )
                shoulder_center_coords = (
                    (left_shoulder_coords[0] + right_shoulder_coords[0]) / 2,
                    (left_shoulder_coords[1] + right_shoulder_coords[1]) / 2
                )

                left_shoulder_coords_int = (int(left_shoulder_coords[0]), int(left_shoulder_coords[1]))
                right_shoulder_coords_int = (int(right_shoulder_coords[0]), int(right_shoulder_coords[1]))
                nose_coords_int = (int(nose_coords[0]), int(nose_coords[1]))
                shoulder_center_coords_int = (int(shoulder_center_coords[0]), int(shoulder_center_coords[1]))

                cv2.circle(frame, left_shoulder_coords_int, radius=5, color=(0, 255, 0), thickness=-1)
                cv2.circle(frame, right_shoulder_coords_int, radius=5, color=(0, 255, 0), thickness=-1)
                cv2.circle(frame, nose_coords_int, radius=5, color=(255, 0, 0), thickness=-1)
                cv2.circle(frame, shoulder_center_coords_int, radius=5, color=(0, 0, 255), thickness=-1)

                cv2.line(frame, left_shoulder_coords_int, right_shoulder_coords_int, color=(255, 255, 255), thickness=2)
                cv2.line(frame, nose_coords_int, shoulder_center_coords_int, color=(255, 255, 255), thickness=2)

                horizontal_angle = calculate_angle(left_shoulder_coords, shoulder_center_coords, nose_coords)
                vertical_angle = calculate_angle(left_shoulder_coords, nose_coords, right_shoulder_coords)

                calibration_data[direction].append((horizontal_angle, vertical_angle))

            overlay = frame.copy()
            cv2.rectangle(overlay, (10, 10), (355, 100), (0, 0, 0), thickness=cv2.FILLED)
            cv2.addWeighted(overlay, 0.5, frame, 0.5, 0, frame)
            draw_countdown_progress_bar(frame, 25, 55, 315, 30, time_difference, 5, (0, 255, 0), 0.65)
            cv2.putText(frame, f"DIRECCION - {direction.upper()}", (20, 40), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
            cv2.imshow("Calibration", frame)

            if cv2.waitKey(1) & 0xFF == 27:
                break
        
    for direction, angles in calibration_data.items():
        if angles:
            horizontal_avg = np.mean([angle[0] for angle in angles])
            vertical_avg = np.mean([angle[1] for angle in angles])
            calibration_results[direction] = (horizontal_avg, vertical_avg)

    play_sound("done.mp3")
    print("Calibration completed:")
    cv2.destroyWindow("Calibration")
    for direction, (h_avg, v_avg) in calibration_results.items():
        print(f"{direction.capitalize()}: Horizontal Avg = {h_avg:.2f}, Vertical Avg = {v_avg:.2f}")

    return calibration_results

def get_angle_color(angle):
    if angle < 45:
        return (0, 255, 0)
    elif 45 <= angle <= 90:
        return (255, 255, 0)
    else:
        return (0, 0, 255)

def draw_countdown_progress_bar(frame, x, y, width, height, seconds_remaining, total_seconds, color, text_font):
    cv2.rectangle(frame, (x, y), (x + width, y + height), (50, 50, 50), -1)
    progress = int(width * (seconds_remaining / total_seconds))
    cv2.rectangle(frame, (x, y), (x + progress, y + height), color, -1)
    cv2.rectangle(frame, (x, y), (x + width, y + height), (255, 255, 255), 2)
    text = f"{int(seconds_remaining)}"
    text_size = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, text_font, 1)[0]
    text_x = x + (width - text_size[0]) // 2
    text_y = y + (height + text_size[1]) // 2
    cv2.putText(frame, text, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, text_font, (255, 255, 255), 2)
    
def draw_progress_bar(frame, x, y, width, height, percentage, color, angle_value):
    cv2.rectangle(frame, (x, y), (x + width, y + height), (50, 50, 50), -1)
    progress = int(width * (percentage / 100))
    cv2.rectangle(frame, (x, y), (x + progress, y + height), color, -1)
    cv2.rectangle(frame, (x, y), (x + width, y + height), (255, 255, 255), 2)
    text = f"{int(angle_value)}"
    text_size = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)[0]
    text_x = x + (width - text_size[0]) // 2
    text_y = y + (height + text_size[1]) // 2
    cv2.putText(frame, text, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)


In [4]:
calibration_results = calibrate_angles()

front_horizontal, front_vertical = calibration_results.get("front", (90, 90))
right_horizontal, right_vertical = calibration_results.get("right", (120, 90))
up_horizontal, up_vertical = calibration_results.get("up", (90, 120))
left_horizontal, left_vertical = calibration_results.get("left", (60, 90))
down_horizontal, down_vertical = calibration_results.get("down", (90, 60))

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(frame_rgb)

    if results.pose_landmarks is not None:
        mp_drawing.draw_landmarks(
            frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(255, 165, 0), thickness=2, circle_radius=3),
            mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=2)
        )

        frame = cv2.flip(frame, 1)
        height, width, _ = frame.shape

        landmarks = results.pose_landmarks.landmark

        right_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER]
        right_elbow = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW]
        right_wrist = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST]

        left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER]
        left_elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW]
        left_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST]

        right_hip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP]
        right_knee = landmarks[mp_pose.PoseLandmark.RIGHT_KNEE]
        right_ankle = landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE]

        left_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP]
        left_knee = landmarks[mp_pose.PoseLandmark.LEFT_KNEE]
        left_ankle = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE]

        nose = landmarks[mp_pose.PoseLandmark.NOSE]

        right_shoulder_coords = (right_shoulder.x * width, right_shoulder.y * height)
        right_elbow_coords = (right_elbow.x * width, right_elbow.y * height)
        right_wrist_coords = (right_wrist.x * width, right_wrist.y * height)

        left_shoulder_coords = (left_shoulder.x * width, left_shoulder.y * height)
        left_elbow_coords = (left_elbow.x * width, left_elbow.y * height)
        left_wrist_coords = (left_wrist.x * width, left_wrist.y * height)

        right_hip_coords = (right_hip.x * width, right_hip.y * height)
        right_knee_coords = (right_knee.x * width, right_knee.y * height)
        right_ankle_coords = (right_ankle.x * width, right_ankle.y * height)

        left_hip_coords = (left_hip.x * width, left_hip.y * height)
        left_knee_coords = (left_knee.x * width, left_knee.y * height)
        left_ankle_coords = (left_ankle.x * width, left_ankle.y * height)

        nose_coords = (nose.x * width, nose.y * height)

        shoulder_center_coords = (
            (left_shoulder_coords[0] + right_shoulder_coords[0]) / 2,
            (left_shoulder_coords[1] + right_shoulder_coords[1]) / 2
        )

        right_arm_angle = calculate_angle(right_shoulder_coords, right_elbow_coords, right_wrist_coords)
        left_arm_angle = calculate_angle(left_shoulder_coords, left_elbow_coords, left_wrist_coords)

        right_leg_angle = calculate_angle(right_hip_coords, right_knee_coords, right_ankle_coords)
        left_leg_angle = calculate_angle(left_hip_coords, left_knee_coords, left_ankle_coords)

        nose_vertical_angle = calculate_angle(left_shoulder_coords, nose_coords, right_shoulder_coords)

        nose_horizontal_angle = calculate_angle(left_shoulder_coords, shoulder_center_coords, nose_coords)

        overlay = frame.copy()
        cv2.rectangle(overlay, (10, 10), (355, 235), (0, 0, 0), thickness=cv2.FILLED)
        cv2.addWeighted(overlay, 0.5, frame, 0.5, 0, frame)

        cv2.putText(frame, "Angulo Brazo Derecho: ", (20, 40), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.55, get_angle_color(right_arm_angle), 2)
        cv2.putText(frame, "Angulo Brazo Izquierdo: ", (20, 70), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.55, get_angle_color(left_arm_angle), 2)
        cv2.putText(frame, "Angulo Pierna Derecha: ", (20, 100), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.55, get_angle_color(right_leg_angle), 2)
        cv2.putText(frame, "Angulo Pierna Izquierda: ", (20, 130), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.55, get_angle_color(left_leg_angle), 2)
        cv2.putText(frame, "Angulo Nariz Horizontal: ", (20, 160), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.55, get_angle_color(left_leg_angle), 2)
        cv2.putText(frame, "Angulo Nariz Vertical: ", (20, 190), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.55, get_angle_color(left_leg_angle), 2)
        
        if not record_events:
            cv2.putText(frame, "Presiona 'ENTER' para empezar", (20, 220), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.55, (0,255,255), 2)
        else:
            cv2.putText(frame, "Presiona 'ESCAPE' para detener", (20, 220), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.55, (0,255,255), 2)

        draw_progress_bar(frame, 240, 25, 100, 20, min(right_arm_angle / 180 * 100, 100), get_angle_color(right_arm_angle), right_arm_angle)
        draw_progress_bar(frame, 240, 55, 100, 20, min(left_arm_angle / 180 * 100, 100), get_angle_color(left_arm_angle), left_arm_angle)
        draw_progress_bar(frame, 240, 85, 100, 20, min(right_leg_angle / 180 * 100, 100), get_angle_color(right_leg_angle), right_leg_angle)
        draw_progress_bar(frame, 240, 115, 100, 20, min(left_leg_angle / 180 * 100, 100), get_angle_color(left_leg_angle), left_leg_angle)
        draw_progress_bar(frame, 240, 145, 100, 20, min(nose_horizontal_angle / 180 * 100, 100), get_angle_color(nose_horizontal_angle), nose_horizontal_angle)
        draw_progress_bar(frame, 240, 175, 100, 20, min(nose_vertical_angle / 180 * 100, 100), get_angle_color(nose_vertical_angle), nose_vertical_angle)

        if record_events:
            if right_arm_angle <= 45 and left_arm_angle <= 45:
                keyboard.press("w")
                mouse.release(pynput.mouse.Button.left)
                mouse.release(pynput.mouse.Button.right)
            else:
                keyboard.release("w")
                if right_arm_angle <= 45:
                    mouse.press(pynput.mouse.Button.left)
                else:
                    mouse.release(pynput.mouse.Button.left)

                if left_arm_angle <= 45:
                    mouse.press(pynput.mouse.Button.right)
                else:
                    mouse.release(pynput.mouse.Button.right)

            if nose_horizontal_angle < left_horizontal:
                mouse.move(-40, 0)
            elif nose_horizontal_angle > right_horizontal:
                mouse.move(40, 0)

            if nose_vertical_angle < up_vertical:
                mouse.move(0, -40)
            elif nose_vertical_angle > down_vertical:
                mouse.move(0, 40)
            
            if right_leg_angle <= 60:
                keyboard.press(pynput.keyboard.Key.space)
            else:
                keyboard.release(pynput.keyboard.Key.space)
            
            if left_leg_angle <= 60:
                keyboard.press(pynput.keyboard.Key.shift_l)
            else:
                keyboard.release(pynput.keyboard.Key.shift_l)

        if cv2.waitKey(1) & 0xFF == 13 and not countdown_started:
            countdown_started = True
            countdown_start_time = time.time()

        if countdown_started:
            elapsed_time = time.time() - countdown_start_time
            if elapsed_time < 5:
                cv2.putText(frame, f"Activando: {int(5 - elapsed_time)}", 
                            (width - 250, height - 30), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            else:
                record_events = not record_events
                countdown_started = False

    cv2.imshow("Frame", frame)

    if cv2.waitKey(1) == 27:
        break

cap.release()
cv2.destroyAllWindows()

Presiona 'SPACE' para comenzar la cuenta atrás.
Calibration completed:
Front: Horizontal Avg = 87.81, Vertical Avg = 91.83
Right: Horizontal Avg = 118.80, Vertical Avg = 81.38
Up: Horizontal Avg = 88.86, Vertical Avg = 70.00
Left: Horizontal Avg = 65.26, Vertical Avg = 66.47
Down: Horizontal Avg = 64.10, Vertical Avg = 123.60
